# Introduction to DataFrames
**[Bogumił Kamiński](http://bogumilkaminski.pl/about/), August 26, 2019**

Let's get started by loading the `DataFrames` package.

In [1]:
using Pkg
Pkg.activate(".");

Activating environment at `d:\Dev\Julia\DataFrames_Tutorial\Project.toml`


In [2]:
using DataFrames, Random

## Constructors and conversion

### Constructors

In this section, you'll see many ways to create a `DataFrame` using the `DataFrame()` constructor.

First, we could create an empty DataFrame,

In [3]:
DataFrame()

""
""


Or we could call the constructor using keyword arguments to add columns to the `DataFrame`.

In [4]:
DataFrame(A=1:3, B=rand(3), C=randstring.([3,3,3]))

,A,B,C
,Int64,Float64,String
1,1,0.41203,LE9
2,2,0.675383,gq0
3,3,0.303751,Eww


We can create a `DataFrame` from a dictionary, in which case keys from the dictionary will be sorted to create the `DataFrame` columns.

In [5]:
x = Dict("A" => [1,2], "B" => [true, false], "C" => ['a', 'b'])
DataFrame(x)

,A,B,C
,Int64,Bool,Char
1,1,1,'a'
2,2,0,'b'


Rather than explicitly creating a dictionary first, as above, we could pass `DataFrame` arguments with the syntax of dictionary key-value pairs. 

Note that in this case, we use symbols to denote the column names and arguments are not sorted. For example, `:A`, the symbol, produces `A`, the name of the first column here:

In [6]:
DataFrame(:A => [1,2], :B => [true, false], :C => ['a', 'b'])

,A,B,C
,Int64,Bool,Char
1,1,1,'a'
2,2,0,'b'


Here we create a `DataFrame` from a vector of vectors, and each vector becomes a column.

In [7]:
DataFrame([rand(3) for i in 1:3])

,x1,x2,x3
,Float64,Float64,Float64
1,0.537589,0.988902,0.93903
2,0.449803,0.287668,0.432202
3,0.123665,0.963943,0.401512


It is not allowed to pass a vector of scalars to `DataFrame` constructor.

In [8]:
DataFrame(rand(3))

ArgumentError: ArgumentError: 'Array{Float64,1}' iterates 'Float64' values, which don't satisfy the Tables.jl Row-iterator interface

Instead use a transposed vector if you have a vector of atoms (in this way you effectively pass a two dimensional array to the constructor which is supported).

In [9]:
DataFrame(permutedims([1, 2, 3]))

,x1,x2,x3
,Int64,Int64,Int64
1,1,2,3


or pass a vector of `NamedTuple`s

In [10]:
v = [(a=1, b=2), (a=3, b=4)]
DataFrame(v)

,a,b
,Int64,Int64
1,1,2
2,3,4


Pass a second argument to give the columns names in case you pass a vector of vectors.

In [11]:
DataFrame([1:3, 4:6, 7:9], [:A, :B, :C])

,A,B,C
,Int64,Int64,Int64
1,1,4,7
2,2,5,8
3,3,6,9


Alternatively you can pass a `NamedTuple` of vectors:

In [12]:
n = (a=1:3, b=11:13)
DataFrame(n)

,a,b
,Int64,Int64
1,1,11
2,2,12
3,3,13


Here we create a `DataFrame` from a matrix,

In [13]:
DataFrame(rand(3,4))

,x1,x2,x3,x4
,Float64,Float64,Float64,Float64
1,0.0479324,0.345159,0.768023,0.297724
2,0.771549,0.868533,0.921015,0.665682
3,0.0507032,0.277992,0.710824,0.500623


and here we do the same but also pass column names.

In [14]:
DataFrame(rand(3,4), Symbol.('a':'d'))

,a,b,c,d
,Float64,Float64,Float64,Float64
1,0.38361,0.900459,0.678818,0.555364
2,0.573168,0.74093,0.589018,0.00700241
3,0.657621,0.805023,0.0265934,0.0755003


We can also construct an uninitialized DataFrame.

Here we pass column types, names and number of rows; we get `missing` in column :C because `Any >: Missing`.

In [15]:
DataFrame([Int, Float64, Any], [:A, :B, :C], 1)

,A,B,C
,Int64,Float64,Any
1,1,1.42526e-315,missing


Here we create a `DataFrame` where `:C` is `#undef`

In [16]:
DataFrame([Int, Float64, String], [:A, :B, :C], 1)

,A,B,C
,Int64,Float64,String
1,4294967296,1.50908e-315,#undef


To initialize a `DataFrame` with column names, but no rows use

In [17]:
DataFrame([Int, Float64, String], [:A, :B, :C])

,A,B,C
,Int64,Float64,String


or

In [18]:
DataFrame(A=Int[], B=Float64[], C=String[])

,A,B,C
,Int64,Float64,String


Finally, we can create a `DataFrame` by copying an existing `DataFrame`.

Note that `copy` also copies the vectors.

In [19]:
x = DataFrame(a=1:2, b='a':'b')
y = copy(x)
(x === y), isequal(x, y), (x.a == y.a), (x.a === y.a)

(false, true, true, false)

Calling `DataFrame` on a `DataFrame` object works like `copy`.

In [20]:
x = DataFrame(a=1:2, b='a':'b')
y = DataFrame(x)
(x === y), isequal(x, y), (x.a == y.a), (x.a === y.a)

(false, true, true, false)

You can avoid copying of columns of a data frame by passing `copycols=false` keyword argument or using `DataFrame!` constructor.

In [21]:
x = DataFrame(a=1:2, b='a':'b')
y = DataFrame(x, copycols=false)
(x === y), isequal(x, y), (x.a == y.a), (x.a === y.a)

(false, true, true, true)

In [22]:
x = DataFrame(a=1:2, b='a':'b')
y = DataFrame!(x)
(x === y), isequal(x, y), (x.a == y.a), (x.a === y.a)

(false, true, true, true)

The same rule applies to other constructors

In [23]:
a = [1, 2, 3]
df1 = DataFrame(a=a)
df2 = DataFrame(a=a, copycols=false)
df1.a === a, df2.a === a

(false, true)

You can create a similar uninitialized `DataFrame` based on an original one:

In [24]:
similar(x)

,a,b
,Int64,Char
1,321237360,'\x12\x77\xe5\x90'
2,321231120,'\0'


In [25]:
similar(x, 0) # number of rows in a new DataFrame passed as a second argument

,a,b
,Int64,Char


You can also create a new `DataFrame` from `SubDataFrame` or `DataFrameRow` (discussed in detail later in the tutorial)

In [26]:
sdf = view(x, [1,1], :)

,a,b
,Int64,Char
1,1,'a'
2,1,'a'


In [27]:
typeof(sdf)

SubDataFrame{DataFrame,DataFrames.Index,Array{Int64,1}}

In [28]:
DataFrame(sdf)

,a,b
,Int64,Char
1,1,'a'
2,1,'a'


In [29]:
dfr = x[1, :]

,a,b
,Int64,Char
1,1,'a'


In [30]:
DataFrame(dfr)

,a,b
,Int64,Char
1,1,'a'


### Conversion to a matrix

Let's start by creating a `DataFrame` with two rows and two columns.

In [31]:
x = DataFrame(x=1:2, y=["A", "B"])

,x,y
,Int64,String
1,1,A
2,2,B


We can create a matrix by passing this `DataFrame` to `Matrix`.

In [32]:
Matrix(x)

2×2 Array{Any,2}:
 1  "A"
 2  "B"

This would work even if the `DataFrame` had some `missing`s:

In [33]:
x = DataFrame(x=1:2, y=[missing,"B"])

,x,y
,Int64,String⍰
1,1,missing
2,2,B


In [34]:
Matrix(x)

2×2 Array{Any,2}:
 1  missing
 2  "B"    

In the two previous matrix examples, Julia created matrices with elements of type `Any`. We can see more clearly that the type of matrix is inferred when we pass, for example, a `DataFrame` of integers to `Matrix`, creating a 2D `Array` of `Int64`s:

In [35]:
x = DataFrame(x=1:2, y=3:4)

,x,y
,Int64,Int64
1,1,3
2,2,4


In [36]:
Matrix(x)

2×2 Array{Int64,2}:
 1  3
 2  4

In this next example, Julia correctly identifies that `Union` is needed to express the type of the resulting `Matrix` (which contains `missing`s).

In [37]:
x = DataFrame(x=1:2, y=[missing,4])

,x,y
,Int64,Int64⍰
1,1,missing
2,2,4


In [38]:
Matrix(x)

2×2 Array{Union{Missing, Int64},2}:
 1   missing
 2  4       

Note that we can't force a conversion of `missing` values to `Int`s!

In [39]:
Matrix{Int}(x)

ArgumentError: ArgumentError: cannot convert a DataFrame containing missing values to Matrix{Int64} (found for column y)

### Conversion to `NamedTuple` related tabular structures

In [40]:
x = DataFrame(x=1:2, y=["A", "B"])

,x,y
,Int64,String
1,1,A
2,2,B


In [41]:
using Tables

First we convert a `DataFrame` into a `NamedTuple` of vectors

In [42]:
ct = Tables.columntable(x)

(x = [1, 2], y = ["A", "B"])

Next we convert it into a vector of `NamedTuples`

In [43]:
rt = Tables.rowtable(x)

2-element Array{NamedTuple{(:x, :y),Tuple{Int64,String}},1}:
 (x = 1, y = "A")
 (x = 2, y = "B")

We can perform the conversions back to a `DataFrame` using a standard constructor call:

In [44]:
DataFrame(ct)

,x,y
,Int64,String
1,1,A
2,2,B


In [45]:
DataFrame(rt)

,x,y
,Int64,String
1,1,A
2,2,B


### Handling of duplicate column names

We can pass the `makeunique` keyword argument to allow passing duplicate names (they get deduplicated)

In [46]:
df = DataFrame(:a=>1, :a=>2, :a_1=>3; makeunique=true)

,a,a_2,a_1
,Int64,Int64,Int64
1,1,2,3


Otherwise, duplicates are not allowed.

In [47]:
df = DataFrame(:a=>1, :a=>2, :a_1=>3)

ArgumentError: ArgumentError: Duplicate variable names: :a. Pass makeunique=true to make them unique using a suffix automatically.

Finallly observe that `nothing` is not printed when displaying a `DataFrame`:

In [48]:
DataFrame(x=[1, nothing], y=[nothing, "a"])

,x,y
,Union…,Union…
1,1,
2,,a
